In [52]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
import pyodbc
conn = pyodbc.connect( DRIVER="{ODBC Driver 17 for SQL Server}",
                       SERVER="127.0.0.1",
                       DATABASE="master",
                       UID="sa",
                       PWD="Qwertyui123")

spark = SparkSession.builder\
    .master("local[*]")\
    .appName("spark_test")\
    .config('spark.driver.extraClassPath', r'C:\spark\spark-3.1.3-bin-hadoop3.2\jars\mssql-jdbc-8.2.2.jre8.jar')\
    .config('spark.executor.extraClassPath', r'C:\spark\spark-3.1.3-bin-hadoop3.2\jars\mssql-jdbc-8.2.2.jre8.jar')\
    .getOrCreate()   
#Формируем df source
stg_acc = spark.read.format("jdbc")\
    .option("url", "jdbc:sqlserver://localhost:1433;databaseName={master}") \
    .option("dbtable", "etl_stg.STG_ACCOUNT") \
    .option("user", "sa") \
    .option("password", "Qwertyui123") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
stg=stg_acc.drop('ORGANISATION_ID')
#Получаем последний идентификатор
max = spark.read.format("jdbc")\
    .option("url", "jdbc:sqlserver://localhost:1433;databaseName={master}") \
    .option("dbtable", "(select case when MAX_SK is null then 0 else MAX_SK end  as MAX_SK from etl_int.MAX_DWH_SK where TABLE_NAME='DIM_ACCOUNT') as Tbl")\
    .option("user", "sa") \
    .option("password", "Qwertyui123") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
#Формируем df target
dwh = spark.read.format("jdbc")\
    .option("url", "jdbc:sqlserver://localhost:1433;databaseName={master}") \
    .option("dbtable", "dwh.DIM_ACCOUNT")\
    .option("user", "sa") \
    .option("password", "Qwertyui123") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
#Формируем темповые вью для sparkSQL
stg.createOrReplaceTempView("stg")
max.createOrReplaceTempView("max")
dwh.createOrReplaceTempView("dwh")
#Формируем temp (join source&target)
tmp=spark.sql("select case when DWH.ACCOUNT_SK is null then (select MAX_SK from MAX) + row_number() over(order by DWH.ACCOUNT_SK)else DWH.ACCOUNT_SK end as ACCOUNT_DIM_SK,STG.ID,STG.ACCOUNT_NUMBER,STG.TYPE,STG.BONUS,STG.QUALIFICATION,STG.CURRENT_LEVEL,STG.STATUS,DATE(STG.CREATED_DATE) as CREATED_DATE,DATE(STG.CLOSED_DATE) as CLOSED_DATE,STG.SWINDLER,STG.IS_LEVEL_CONFIRMED,DATE(STG.NEGATIVE_BALANCE_DATE) as NEGATIVE_BALANCE_DATE,STG.BONUS_FPK,STG.BONUS_PARTNER,STG.MERGED_ACTIVE_ACCOUNT_ID from stg left join dwh on stg.ACCOUNT_NUMBER = dwh.ACCOUNT_BK1")
#Очищаем temp
df=conn.cursor()
df.execute("truncate table etl_int.TMP_ACCOUNT;")
conn.commit()
#Пишем temp в БД
tmp.write.format('jdbc')\
    .option("url", "jdbc:sqlserver://localhost:1433;databaseName={master}") \
    .option("dbtable", "etl_int.tmp_ACCOUNT") \
    .option("user", "sa") \
    .option("password", "Qwertyui123") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .mode('append').save()
#Запускаем мердж temp и target
df2=conn.cursor() 
df2.execute("""
MERGE dwh.DIM_ACCOUNT AS Target
USING etl_int.TMP_ACCOUNT	AS source
ON (Source.ACCOUNT_DIM_SK = Target.ACCOUNT_SK)
WHEN MATCHED THEN UPDATE SET
	target.ACCOUNT_BK1=source.ACCOUNT_NUMBER,
	target.ACCOUNT_BK2=source.ID,
	target.TYPE=source.TYPE,
	target.BONUS=source.BONUS,
	target.QUALIFICATION=source.QUALIFICATION,
	target.CURRENT_LEVEL=source.CURRENT_LEVEL,
	target.STATUS=source.STATUS,
	target.CREATED_DATE=source.CREATED_DATE,
	target.CLOSED_DATE=source.CLOSED_DATE,
	target.SWINDLER=source.SWINDLER,
	target.IS_LEVEL_CONFIRMED=source.IS_LEVEL_CONFIRMED,
	target.NEGATIVE_BALANCE_DATE=source.NEGATIVE_BALANCE_DATE,
	target.BONUS_FPK=source.BONUS_FPK,
	target.BONUS_PARTNER=source.BONUS_PARTNER,
	target.MERGED_ACTIVE_ACCOUNT_ID=source.MERGED_ACTIVE_ACCOUNT_ID	
WHEN NOT MATCHED BY Target THEN
    INSERT (ACCOUNT_SK, ACCOUNT_BK1, ACCOUNT_BK2, [TYPE], BONUS, QUALIFICATION, CURRENT_LEVEL, STATUS, CREATED_DATE, CLOSED_DATE, SWINDLER, IS_LEVEL_CONFIRMED, NEGATIVE_BALANCE_DATE, BONUS_FPK, BONUS_PARTNER, MERGED_ACTIVE_ACCOUNT_ID) 
    VALUES (source.ACCOUNT_DIM_SK,source.ACCOUNT_NUMBER,source.ID,source.[TYPE],source.BONUS,source.QUALIFICATION,source.CURRENT_LEVEL,source.STATUS,source.CREATED_DATE,source.CLOSED_DATE,source.SWINDLER,source.IS_LEVEL_CONFIRMED,source.NEGATIVE_BALANCE_DATE,source.BONUS_FPK,source.BONUS_PARTNER,source.MERGED_ACTIVE_ACCOUNT_ID)
    ;
    """
)
conn.commit()  
conn.close()

C:\Users\DBliznets\AppData\Local\Temp\ipykernel_18856\600535478.py:5: DeprecationWarning: PyUnicode_FromUnicode(NULL, size) is deprecated; use PyUnicode_New() instead
  conn = pyodbc.connect( DRIVER="{ODBC Driver 17 for SQL Server}",
